# Immigration datawarehousing project
### Data Engineering Capstone Project

#### Project Summary
In this project, we will demonstrate how to build a data warehouse that enriches the analysis of frequently occurring events by augmenting the event data with ancillary information. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import configparser
import psycopg2
from sql_queries import airport_insert, demographic_insert, immigration_insert, temperature_insert

### Step 1: Scope the Project and Gather Data

#### Scope 
We explain the scope of this project in detail:

- To aid in the analysis of U.S. immigration data, we expand the dataset with information on the ports through which immigration occurs.
  - This information includes data on airports, demographics, and temperatures.
- We will finally construct the data warehouse in snowflake schema by building ETL processes which convert original data into these tables.
- We use pandas and PostgresSQL in local development.

#### Describe and Gather Data 
We use the following datasets:
- **I94 Immigration Data**: This data comes from [the US National Tourism and Trade Office](https://www.trade.gov/national-travel-and-tourism-office).
  - A data dictionary exported from SAS is included in the repository: I94_SAS_Labels_Descriptions.SAS.
- **Airport Code Table**: This is a simple table of airport codes and corresponding cities published [here](https://datahub.io/core/airport-codes#data).
- **U.S. City Demographic Data**: This data comes from [OpenSoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).
- **World Temperature Data**: This dataset came from Kaggle kernel: [Climate Change: Earth Surface Temperature Data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).


In [2]:
df_immigration = pd.read_csv('immigration_data_sample.csv')
print(df_immigration.shape)
df_immigration = df_immigration.set_index('Unnamed: 0').sort_index()
df_immigration.head()

(1000, 29)


,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
10925,13208.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20574.0,...,NaN,M,1987.0,06292016,M,NaN,VS,5.544224e+10,00007,WT
10930,13213.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20553.0,...,NaN,M,1981.0,06292016,NaN,NaN,AA,5.544979e+10,00109,WT
11328,13826.0,2016.0,4.0,117.0,117.0,ATL,20545.0,1.0,SC,20553.0,...,NaN,M,1972.0,06292016,M,NaN,AF,5.545908e+10,00688,WB
14575,17786.0,2016.0,4.0,123.0,123.0,NYC,20545.0,1.0,NE,20556.0,...,NaN,M,1985.0,06292016,NaN,NaN,VS,5.545518e+10,00009,WB
15053,18310.0,2016.0,4.0,123.0,123.0,SEA,20545.0,1.0,CA,20548.0,...,NaN,M,1971.0,06292016,M,NaN,DL,5.542154e+10,00143,WT


In [3]:
df_airport = pd.read_csv('airport-codes_csv.csv')
print(df_airport.shape)
df_airport.head()

(55075, 12)


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [4]:
df_demographics = pd.read_csv('us-cities-demographics.csv', delimiter=';')
print(df_demographics.shape)
df_demographics.head()

(2891, 12)


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [5]:
df_temperature = pd.read_csv('GlobalLandTemperaturesByCity.csv')
print(df_temperature.shape)
df_temperature_us = df_temperature[df_temperature["Country"] == "United States"]
print(df_temperature_us.shape)
df_temperature_us.head()

(8599212, 7)
(687289, 7)


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [6]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.\
# config("spark.jars.repositories", "https://repos.spark-packages.org/").\
# config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
# enableHiveSupport().getOrCreate()

# df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
# #write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

In [7]:
print("Define valid iata codes, cities and state codes from the SAS description file.")
iata_codes, cities, state_codes = [], [], []

with open("./I94_SAS_Labels_Descriptions.SAS") as f:
    lines = [l.strip() for l in f]
    print(lines[300:302], lines[962:963])
    for port in lines[302:962]:
        iata_code, city_state = tuple(map(lambda x:x.replace("'","").strip(), port.split("=")))
        if len(city_state.rsplit(',', 1)) == 2:
            city, state_code = tuple(map(lambda x: x.strip(), city_state.rsplit(',', 1)))
            iata_codes.append(iata_code)
            cities.append(city)
            state_codes.append(state_code)

Define valid iata codes, cities and state codes from the SAS description file.
['/* I94PORT - This format shows all the valid and invalid codes for processing */', 'value $i94prtl'] [';']


In [8]:
print("Identify columns which have too many NaNs.")
# key: i94port, i94addr
["visapost", "occup", "entdepu", "insnum", ]
for k in [k for k,v in df_immigration.isnull().any().items() if v]:
    print(df_immigration[k].isna().value_counts().sort_index())

Identify columns which have too many NaNs.
False    941
True      59
Name: i94addr, dtype: int64
False    951
True      49
Name: depdate, dtype: int64
False    382
True     618
Name: visapost, dtype: int64
False      4
True     996
Name: occup, dtype: int64
False    954
True      46
Name: entdepd, dtype: int64
True    1000
Name: entdepu, dtype: int64
False    954
True      46
Name: matflag, dtype: int64
False    859
True     141
Name: gender, dtype: int64
False     35
True     965
Name: insnum, dtype: int64
False    967
True      33
Name: airline, dtype: int64
False    992
True       8
Name: fltno, dtype: int64


In [9]:
print("Identify columns which have too many NaNs.")
# key: iata_code
["continent", "local_code",]
for k in [k for k,v in df_airport.isnull().any().items() if v]:
    print(df_airport[k].isna().value_counts().sort_index())

Identify columns which have too many NaNs.
False    48069
True      7006
Name: elevation_ft, dtype: int64
False    27356
True     27719
Name: continent, dtype: int64
False    54828
True       247
Name: iso_country, dtype: int64
False    49399
True      5676
Name: municipality, dtype: int64
False    41030
True     14045
Name: gps_code, dtype: int64
False     9189
True     45886
Name: iata_code, dtype: int64
False    28686
True     26389
Name: local_code, dtype: int64


In [10]:
print("Identify duplicated key columns.")
_df_airport = df_airport.dropna(subset=['iata_code'])
_df_airport = _df_airport[_df_airport["iata_code"].isin(iata_codes)]
# _df_airport[]
_df_airport = _df_airport[_df_airport['type']!='closed']
# _df_airport.drop_duplicates(subset=['iata_code']).shape
_df_airport[_df_airport.duplicated(subset=['iata_code'])]

Identify duplicated key columns.


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
40590,RU-0493,small_airport,Dalnerechensk Airport,272.0,EU,RU,RU-PRI,Dalnerechensk,NaN,DLR,NaN,"133.7363, 45.8783"


In [11]:
print("Identify columns which have too many NaNs.")
# key: City, State Code
for k in [k for k,v in df_demographics.isnull().any().items() if v]:
    print(df_demographics[k].isna().value_counts().sort_index())

Identify columns which have too many NaNs.
False    2888
True        3
Name: Male Population, dtype: int64
False    2888
True        3
Name: Female Population, dtype: int64
False    2878
True       13
Name: Number of Veterans, dtype: int64
False    2878
True       13
Name: Foreign-born, dtype: int64
False    2875
True       16
Name: Average Household Size, dtype: int64


In [12]:
print("Identify duplicated key columns.")
_df_demographics = df_demographics.dropna(subset=['City'])
_df_demographics = _df_demographics[_df_demographics["State Code"].isin(state_codes)]
print("Note that demographic data has duplicate rows on (city, state) with different the race values.")
_df_demographics[_df_demographics.duplicated(subset=['City', 'State', 'Race'])] #.head(20)

Identify duplicated key columns.
Note that demographic data has duplicate rows on (city, state) with different the race values.


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count


In [13]:
# key: City
print("Note that temperature data has duplicate rows on city with different (latitude, longitude) values.")
df_temperature_us[df_temperature_us.duplicated(subset=['dt', 'City', 'Latitude', 'Longitude'])]

Note that temperature data has duplicate rows on city with different (latitude, longitude) values.


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude


#### Cleaning Steps
Document steps necessary to clean the data

In [14]:
print(df_immigration.shape)
print("Filter rows which have valid port and address values.")
df_immigration_filtered = df_immigration[df_immigration["i94port"].isin(iata_codes)]
print(df_immigration_filtered.shape)
df_immigration_filtered = df_immigration_filtered[df_immigration_filtered["i94addr"].isin(state_codes)]
print(df_immigration_filtered.shape)
print("Drop columns which have too many NaNs, then drop rows with any NaNs.")
df_immigration_dropped = df_immigration_filtered.drop(columns=["insnum", "entdepu", "occup", "visapost"])
df_immigration_notnull = df_immigration_dropped.dropna()
print(df_immigration_notnull.shape)

df_immigration_notnull.isnull().any()

(1000, 28)
Filter rows which have valid port and address values.
(973, 28)
(904, 28)
Drop columns which have too many NaNs, then drop rows with any NaNs.
(726, 24)


cicid       False
i94yr       False
i94mon      False
i94cit      False
i94res      False
i94port     False
arrdate     False
i94mode     False
i94addr     False
depdate     False
i94bir      False
i94visa     False
count       False
dtadfile    False
entdepa     False
entdepd     False
matflag     False
biryear     False
dtaddto     False
gender      False
airline     False
admnum      False
fltno       False
visatype    False
dtype: bool

In [15]:
print("Filter rows which have valid port and address values.")
df_airport_filtered = df_airport.dropna(subset=['iata_code'])
print(df_airport_filtered.shape)
df_airport_filtered = df_airport_filtered[df_airport_filtered["iata_code"].isin(iata_codes)]
print(df_airport_filtered.shape)
df_airport_filtered = df_airport_filtered[df_airport_filtered['type']!='closed']
df_airport_filtered = df_airport_filtered.drop_duplicates(subset=['iata_code'])
print(df_airport_filtered.shape)

print("Identify columns which have too many NaNs.")
["continent", "local_code", ]
for k in [k for k,v in df_airport_filtered.isnull().any().items() if v]:
    print(df_airport_filtered[k].isna().value_counts().sort_index())
print("Drop columns which have too many NaNs, then drop rows with any NaNs.")
df_airport_dropped = df_airport_filtered.drop(columns=["continent", "local_code", ])
df_airport_notnull = df_airport_dropped.dropna()
print(df_airport_notnull.shape)
df_airport_notnull.isnull().any()

Filter rows which have valid port and address values.
(9189, 12)
(510, 12)
(495, 12)
Identify columns which have too many NaNs.
False    479
True      16
Name: elevation_ft, dtype: int64
False    220
True     275
Name: continent, dtype: int64
False    494
True       1
Name: iso_country, dtype: int64
False    468
True      27
Name: municipality, dtype: int64
False    476
True      19
Name: gps_code, dtype: int64
False    283
True     212
Name: local_code, dtype: int64
Drop columns which have too many NaNs, then drop rows with any NaNs.
(440, 10)


ident           False
type            False
name            False
elevation_ft    False
iso_country     False
iso_region      False
municipality    False
gps_code        False
iata_code       False
coordinates     False
dtype: bool

In [17]:
print(df_demographics.shape)
print("Filter rows which have valid state code values.")
df_demographics_filtered = df_demographics[df_demographics["State Code"].isin(state_codes)]
print(df_demographics_filtered.shape)
print("Drop columns which have any NaNs.")
df_demographics_notnull = df_demographics_filtered.dropna()
print(df_demographics_notnull.shape)
df_demographics_notnull.isnull().any()

(2891, 12)
Filter rows which have valid state code values.
(2886, 12)
Drop columns which have any NaNs.
(2870, 12)


City                      False
State                     False
Median Age                False
Male Population           False
Female Population         False
Total Population          False
Number of Veterans        False
Foreign-born              False
Average Household Size    False
State Code                False
Race                      False
Count                     False
dtype: bool

In [18]:
print(df_temperature_us.shape)
print("Filter rows to the last five years.")
df_temperature_us_copied = df_temperature_us.copy()
df_temperature_us_copied.dt = pd.to_datetime(df_temperature_us_copied.dt)
df_temperature_us_last = df_temperature_us_copied.set_index('dt').sort_index().last('5Y')
df_temperature_us_last = df_temperature_us_last.reset_index()
print(df_temperature_us_last.shape)
print("Filter rows which have valid city values.")
cities_l = [city.lower() for city in cities]
df_temperature_us_filtered = df_temperature_us_last[df_temperature_us_last["City"].str.lower().isin(cities_l)]
print(df_temperature_us_filtered.shape)
print("Drop columns which have any NaNs.")
df_temperature_us_notnull = df_temperature_us_filtered.dropna()
print(df_temperature_us_notnull.shape)
df_temperature_us_notnull.isnull().any()

(687289, 7)
Filter rows to the last five years.
(14649, 7)
Filter rows which have valid city values.
(5244, 7)
Drop columns which have any NaNs.
(5243, 7)


dt                               False
AverageTemperature               False
AverageTemperatureUncertainty    False
City                             False
Country                          False
Latitude                         False
Longitude                        False
dtype: bool

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

We chose the data model in the following table and the reasons are following:
- We take immigration data as the fact table because this is frequently generated event data.
- We take airports, demographics and temperature data as the dimension tables because they can be joined via airport location keys, including iata code, city name and state code.


| table name | description | columns |
| ------- | ---------- | ----------- |
| immigrations | A fact table which stores all immigrations data. | \|cicid\|year\|month\|cit\|res\|iata\|arrdate\|mode\|addr\|depdate\|bir\|visa\|coun\|dtadfil\|visapost\|occup\|entdepa\|entdepd\|entdepu\|matflag\|biryear\|dtaddto\|gender\|insnum\|airline\|admnum\|fltno\|visatype\| |
| airports | A dimension table which stores airport information. | \|iata_code\|name\|type\|coordinates\|city\|state_code\|iso_country\|iso_region\|gps_code\| |
| demographics | A dimension table which stores demographics of cities in the U.S. | \|city\|state_code\|state\|race\|count\|male_population\|female_population\|total_population\|media_age\|num_veterans\|foreign_born\|average_household_size\| |
| temperatures | A dimension table which stores temperature in the U.S. | \|timestamp\|average_temperature\|average_temperatur_uncertainty\|city\|country\|latitude\|longitude\| |

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Join port locations in immigration data to airports data.
2. Rename and sort columns of each table data to match the schema.
3. Create database and each tables by executing create_tables.py.
4. Insert data into created tables.

### Step 4: Run Pipelines to Model the Data 


#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [19]:
df_immigration_port = pd.DataFrame({"iata_code" : iata_codes, "city": cities, "state_code": state_codes})
df_airport_joined = df_airport_notnull.merge(df_immigration_port, left_on="iata_code", right_on="iata_code")
df_airport_model = df_airport_joined[
    [
        "iata_code",
        "name",
        "type",
        "coordinates",
        "city",
        "state_code",
        "iso_country",
        "iso_region",
        "gps_code",
    ]
]

In [20]:
df_demographics_rn = df_demographics_notnull.rename(columns={
    "City": "city",
    "State": "state",
    "Median Age": "median_age",
    "Male Population": "male_population",
    "Female Population": "female_population",
    "Total Population": "total_population",
    "Number of Veterans": "num_veterans",
    "Foreign-born": "foreign_born",
    "Average Household Size": "average_household_size",
    "State Code": "state_code",
    "Race": "race",
    "Count": "count",
})
df_demographics_model = df_demographics_rn[
    [
        "city",
        "state_code",
        "state",
        "race",
        "count",
        "male_population",
        "female_population",
        "total_population",
        "median_age",
        "num_veterans",
        "foreign_born",
        "average_household_size",
    ]
]

In [21]:
# ["insnum", "entdepu", "occup", "visapost"]
df_immigration_notnull_rn = df_immigration_notnull.rename(columns={
    "cicid": "cicid",
    "i94yr": "year",
    "i94mon": "month",
    "i94cit": "cit",
    "i94res": "res",
    "i94port": "iata",
    "arrdate": "arrdate",
    "i94mode": "mode",
    "i94addr": "addr",
    "depdate": "depdate",
    "i94bir": "bir",
    "i94visa": "visa",
    "count": "count",
    "dtadfile": "date_added",
    "entdepa": "entdepa",
    "entdepd": "entdepd",
    "matflag": "matflag",
    "biryear": "biryear",
    "dtaddto": "dtaddto",
    "gender": "gender",
    "airline": "airline",
    "admnum": "admnum",
    "fltno": "fltno",
    "visatype": "visatype",
})

df_immigration_model = df_immigration_notnull_rn[
    [
        "cicid",
        "year",
        "month",
        "cit",
        "res",
        "iata",
        "arrdate",
        "mode",
        "addr",
        "depdate",
        "bir",
        "visa",
        "count",
        "date_added",
        "entdepa",
        "entdepd",
        "matflag",
        "biryear",
        "dtaddto",
        "gender",
        "airline",
        "admnum",
        "fltno",
        "visatype",
    ]
]

In [22]:
df_temperature_us_notnull_rn = df_temperature_us_notnull.rename(columns={
    "dt": "timestamp",
    "AverageTemperature": "average_temperature",
    "AverageTemperatureUncertainty": "average_temperature_uncertainty",
    "City": "city",
    "Country": "country",
    "Latitude": "latitude",
    "Longitude": "longitude",
})
df_temperature_us_model = df_temperature_us_notnull_rn[
    [
        "timestamp",
        "average_temperature",
        "average_temperature_uncertainty",
        "city",
        "country",
        "latitude",
        "longitude",
    ]
]

In [23]:
!python create_tables.py

connecting to postgres...
DROP EXISTING TABLES...
CREATE TABLES...


In [24]:
# Insert data
import configparser
import psycopg2
from sql_queries import airport_insert, demographic_insert, immigration_insert, temperature_insert

config = configparser.ConfigParser()
config.read("db.cfg")
conn = psycopg2.connect(
    "host={} dbname={} user={} password={} port={}".format(
        *config["LOCAL"].values()
    )
)
cur = conn.cursor()


for index, row in df_airport_model.iterrows():
    cur.execute(airport_insert, list(row.values))
    conn.commit()

for index, row in df_demographics_model.iterrows():
    cur.execute(demographic_insert, list(row.values))
    conn.commit()

for index, row in df_immigration_model.iterrows():
    cur.execute(immigration_insert, list(row.values))
    conn.commit()

for index, row in df_temperature_us_model.iterrows():
    cur.execute(temperature_insert, list(row.values))
    conn.commit()

In [26]:
# conn.close()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [27]:
# Perform quality checks here
def assert_count(cur, conn, table):
    """ Assert that the given has any rows.
    Args:
        cur: SQL cursor object.
        conn: DB connection object.
        table: The name of table to check.
    Returns:
    Raises:
        AssertionError
    """
    cur.execute(f"select count(*) from {table}")
    conn.commit()
    assert cur.rowcount >= 1

assert_count(cur, conn, "airports")
assert_count(cur, conn, "demographics")
assert_count(cur, conn, "immigrations")
assert_count(cur, conn, "temperatures")

In [28]:
conn.close()

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up

* Clearly state the rationale for the choice of tools and technologies for the project.
  - We use pandas and PostgresSQL in local development. If large scale analysis is needed, these can be naturally extended to Spark and Redshift for scaling.


* Propose how often the data should be updated and why.
  - We should update the data on a monthly basis since the dataset describes immigration information that is aggregated monthly.


* Write a description of how you would approach the problem differently under the following scenarios:
  * The data was increased by 100x.
    - We should use Spark cluster with EMR for example to scale out ETL process. If SQL querying is slow, use columnar database like Redshift.
  * The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - We sholud use data workflow system like Airflow to schedule our ETL pipeline execution.
  * The database needed to be accessed by 100+ people.
    - We could scale out Redshift cluster to handle a large number of user requests. 
